In [ ]:
#To install pyswarms
!pip install pyswarms

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import pyswarms as ps
import io
import re
import string

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from google.colab import files

In [ ]:
# Load the dataset
uploaded = files.upload()
spam_df = pd.read_csv(io.BytesIO(uploaded['emails.csv']))

Saving emails.csv to emails (2).csv


KeyError: 'emails.csv'

In [48]:
spam_df

,subject,message,label
0,job posting - apple-iss research center,content length 3386 apple iss research center ...,0
1,NaN,lang classification grimes joseph e and barbar...,0
2,query : letter frequencies for text identifica...,i am posting this inquiry for sergei atamas sa...,0
3,risk,a colleague and i are researching the differin...,0
4,request book information,earlier this morning i was on the phone with a...,0
...,...,...,...
2888,love your profile - ysuolvpv,hello thanks for stopping by we have taken man...,1
2889,you have been asked to join kiddin,the list owner of kiddin has invited you to jo...,1
2890,anglicization of composers ' names,judging from the return post i must have sound...,0
2891,"re : 6 . 797 , comparative method : n - ary co...",gotcha there are two separate fallacies in the...,0


In [ ]:
spam_df.head(10)

,subject,message,label
0,job posting - apple-iss research center,content - length : 3386 apple-iss research cen...,0
1,NaN,"lang classification grimes , joseph e . and ba...",0
2,query : letter frequencies for text identifica...,i am posting this inquiry for sergei atamas ( ...,0
3,risk,a colleague and i are researching the differin...,0
4,request book information,earlier this morning i was on the phone with a...,0
5,call for abstracts : optimality in syntactic t...,content - length : 4437 call for papers is the...,0
6,m . a . in scandinavian linguistics,m . a . in scandinavian linguistics at the uni...,0
7,call for papers : linguistics session of the m...,call for papers linguistics session - - midwes...,0
8,foreign language in commercials,content - length : 1937 greetings ! i ' m wond...,0
9,fulbright announcement : please post / dissemi...,fulbright announcement : please post / dissemi...,0


In [ ]:
spam_df.tail(10)

,subject,message,label
2883,evolvable hardware and gp,"paper now available in post script . . . "" rap...",0
2884,did that work for you ? - calsvxtn,hello thanks for stopping by ! ! we have taken...,1
2885,british vs . american < a > ( s,griffin bacal internet mail direct inquiries t...,0
2886,"from fanny , recommending nek","dear sir or madam , this is not a "" spam "" mes...",1
2887,win $ 300usd and a cruise !,"raquel 's casino , inc . is awarding a cruise ...",1
2888,love your profile - ysuolvpv,hello thanks for stopping by ! ! we have taken...,1
2889,you have been asked to join kiddin,"the list owner of : "" kiddin "" has invited you...",1
2890,anglicization of composers ' names,"judging from the return post , i must have sou...",0
2891,"re : 6 . 797 , comparative method : n - ary co...",gotcha ! there are two separate fallacies in t...,0
2892,re : american - english in australia,hello ! i ' m working on a thesis concerning a...,0


In [ ]:
spam_df.describe()

,label
count,2893.000000
mean,0.166263
std,0.372381
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [ ]:
spam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2893 entries, 0 to 2892
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   subject  2831 non-null   object
 1   message  2893 non-null   object
 2   label    2893 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 67.9+ KB


In [ ]:
# Visualize the Data
ham=spam_df[spam_df['label']==0]
spam=spam_df[spam_df['label']==1]

In [ ]:
ham

,subject,message,label
0,job posting - apple-iss research center,content - length : 3386 apple-iss research cen...,0
1,NaN,"lang classification grimes , joseph e . and ba...",0
2,query : letter frequencies for text identifica...,i am posting this inquiry for sergei atamas ( ...,0
3,risk,a colleague and i are researching the differin...,0
4,request book information,earlier this morning i was on the phone with a...,0
...,...,...,...
2883,evolvable hardware and gp,"paper now available in post script . . . "" rap...",0
2885,british vs . american < a > ( s,griffin bacal internet mail direct inquiries t...,0
2890,anglicization of composers ' names,"judging from the return post , i must have sou...",0
2891,"re : 6 . 797 , comparative method : n - ary co...",gotcha ! there are two separate fallacies in t...,0


In [ ]:
spam

,subject,message,label
21,free,this is a multi-part message in mime format . ...,1
38,the internet success toolbox,note : we do not wish to send e-mail to anyone...,1
84,free stealth 3 . 0 bulk email software . . .,"just released . . . 30 , 000 , 000 email addre...",1
85,need more money ?,"hi , would you like to earn an extra $ 700 a w...",1
86,cable decsrambler now only $ 6 . 99 !,this is really cool ! premium channels and pay...,1
...,...,...,...
2884,did that work for you ? - calsvxtn,hello thanks for stopping by ! ! we have taken...,1
2886,"from fanny , recommending nek","dear sir or madam , this is not a "" spam "" mes...",1
2887,win $ 300usd and a cruise !,"raquel 's casino , inc . is awarding a cruise ...",1
2888,love your profile - ysuolvpv,hello thanks for stopping by ! ! we have taken...,1


In [ ]:
print('Spam Percentage =',(len(spam)/len(spam_df))*100,'%')

Spam Percentage = 16.626339440027653 %


In [ ]:
print('Ham Percentage =',(len(ham)/len(spam_df))*100,'%')

Ham Percentage = 83.37366055997235 %


In [ ]:
# functions

# Preprocessing
def preprocess_text(text):
    # Check if the text is not NaN (not a number), if so, convert it to empty string
    if pd.isnull(text):
        text = ''

    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Return the cleaned text
    return text


  # Define objective function for PSO
def f(x, alpha=0.5):
    n_particles = x.shape[0]
    j = [None] * n_particles
    for i in range(n_particles):
        # Set the hyperparameter alpha for the Naive Bayes classifier

        alpha_value = x[i][0] if isinstance(x[i], (list, np.ndarray)) else x[i]

        classifier = MultinomialNB(alpha=alpha_value)
        classifier.fit(X_train, y_train)
        # Use a validation set or cross-validation to avoid overfitting the test set
        j[i] = cross_val_score(classifier, X_train, y_train, cv=5).mean()  # Using cross-validation
    return np.array(j)


# preprocessing
spam_df['message'] = spam_df['message'].apply(preprocess_text)

# Vectorize the text
vectorizer = CountVectorizer(preprocessor=preprocess_text)
X = vectorizer.fit_transform(spam_df['message'])

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, spam_df['label'], test_size=0.3, random_state=42)

# Define the initial Naive Bayes model
nb_model = MultinomialNB()

# Train the model
nb_model.fit(X_train, y_train)

# Predict and evaluate the initial model
y_pred = nb_model.predict(X_test)
print(classification_report(y_test, y_pred))
print("Initial accuracy:", accuracy_score(y_test, y_pred))


# Set-up hyperparameters for PSO
options = {'c1': 1.49618, 'c2': 1.49618, 'w':0.7298, 'k': 20, 'p':2}

# Define bounds for alpha - ensure it's non-negative
# The lower bound is set to a small positive value to avoid alpha being exactly 0
# Adjusting the upper bound as per my requirement
bounds = ([0.01], [20.0])

# Create a PSO instance
optimizer = ps.single.GlobalBestPSO(n_particles=8, dimensions=1, options=options, bounds=bounds)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=50)

# Apply the optimized hyperparameters
nb_optimized = MultinomialNB(alpha=pos[0])
nb_optimized.fit(X_train, y_train)

# Predict and evaluate the optimized model
y_pred_optimized = nb_optimized.predict(X_test)
print(classification_report(y_test, y_pred_optimized))
print("Optimized accuracy:", accuracy_score(y_test, y_pred_optimized))